In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams

In [2]:
train = pd.read_csv('forty_column_dataset.csv')

In [17]:
#new column ideas
#juiceboy = pd.read_csv('juice_test.csv')
juiceboy = pd.read_csv('juice_test_final.csv')

In [18]:
train['civage'] = train['CarInsuredValue']/train['CarAge']
juiceboy['civage'] = juiceboy['CarInsuredValue']/juiceboy['CarAge']

In [5]:
master_mask = np.arange(len(train['labels']))
pos_choices = np.random.choice(master_mask[np.array(train['labels'])==1], size=2.27000000e+04/3)
neg_choices = np.random.choice(master_mask[np.array(train['labels'])==0], size=2.88857000e+06/3)
List = np.concatenate((neg_choices, pos_choices))
train_2 = train.ix[List]

/home/of3/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
/home/of3/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()


In [6]:
train_3 = train_2.drop([u'Unnamed: 0', u'CustomerMD5Key', u'FirstDriverDrivingLicenseNumberY', \
                                   u'CarParkingTypeId',u'FirstDriverDrivingLicenceType',\
                                   u'CarDrivingEntitlement',  u'CarTransmissionId', u'PolicyHolderResidencyArea'\
                        , u'car_flag_1', u'car_flag_4', u'car_flag_3', u'vendor_log_sales', u'RatedDriverNumber',\
                       'CarFuelId','AllDriversNbConvictions'], axis=1)

In [19]:
juiceboy = juiceboy.drop([u'Unnamed: 0', u'CustomerMD5Key', u'FirstDriverDrivingLicenseNumberY', \
                                   u'CarParkingTypeId',u'FirstDriverDrivingLicenceType',\
                                   u'CarDrivingEntitlement',  u'CarTransmissionId', u'PolicyHolderResidencyArea'\
                        , u'car_flag_1', u'car_flag_4', u'car_flag_3', u'vendor_log_sales', u'RatedDriverNumber',\
                       'CarFuelId','AllDriversNbConvictions'], axis=1)

In [20]:
target = 'labels'

In [9]:
xgbl = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

supplied_arr = juiceboy
predictors = [x for x in train_3.columns if x not in [target,'Unnamed: 0', u'CustomerMD5Key']]
dtrain = train_3
alg = xgbl
useTrainCV=True
cv_folds=5
early_stopping_rounds=50
predictors = [x for x in train_3.columns if x not in [target,'Unnamed: 0', u'CustomerMD5Key']]
    
if useTrainCV:
    xgb_param = alg.get_xgb_params()
    xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
    cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
        metrics='auc', early_stopping_rounds=early_stopping_rounds)
    alg.set_params(n_estimators=cvresult.shape[0])

#Fit the algorithm on the data
alg.fit(dtrain[predictors], dtrain['labels'],eval_metric='auc')

#Predict training set:
dtrain_predictions = alg.predict(dtrain[predictors])
dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]

dtrain_predlabels = alg.predict_proba(supplied_arr)


print "\nModel Report"
print "Accuracy : %.4g" % metrics.accuracy_score(dtrain['labels'].values, dtrain_predictions)
print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['labels'], dtrain_predprob)


Model Report
Accuracy : 0.9994
AUC Score (Train): 0.999945


In [10]:
print "Done shiok beta"

Done shiok beta


In [15]:
np.savetxt("shiok_beta.csv", [x[1] for x in dtrain_predlabels], delimiter="\n")

In [11]:
1+1

2

In [16]:
print cvresult.shape[0]

348


In [15]:
import joblib
joblib.dump(alg, 'shiok_beta_classifier.pkl') 

['shiok_beta_classifier.pkl']

In [21]:
supplied_arr = juiceboy

In [22]:
dtrain_predlabels = alg.predict_proba(supplied_arr)

In [27]:
np.savetxt("shiok_beta_final.csv", [x[1] for x in dtrain_predlabels], delimiter="\n")